In [1]:
from itertools import groupby
import pandas as pd

## Алгоритм Кнута-Мориса-Прата

### 1. Работа с файлами формата FASTA 

Реализуем функцию, считывающую файл и создающую итератор, содержащий все последовательности в файле

In [2]:
def fasta_iter(fasta_name):
    """
    Parses a FASTA file.
    Yields tuples of header string and sequence
    """
    fh = open(fasta_name)
    # create iterator object
    faiter = (x[1] for x in groupby(fh, lambda line: line[0] == ">"))
    for header in faiter:
        # drop the ">"
        headerStr = header.__next__()[1:].strip()
        # join all sequence lines to one.
        seq = "".join(s.strip() for s in faiter.__next__())
        yield (headerStr, seq)

Примеры использования:

In [20]:
arabidopsis_iter = fasta_iter('arabidopsis_atc.fasta')

for i in arabidopsis_iter:
    header, arabidopsis_atc_seq = i
 
print(header)
print('First 50 nucleotides:', arabidopsis_atc_seq[:50])

AB024714.1 Arabidopsis thaliana ATC (centroradialis) gene, complete cds, strain:Columbia
First 50 nucleotides: CATAGGTTCGAAATTGGTTTATTATAATTTTAACGCACAAATTCCTTGTT


In [4]:
example_iter = fasta_iter('fasta_example.fasta')
rows = []

for i in example_iter:
    rows.append(i)
    
example_df = pd.DataFrame(rows, columns=['Sequence header', 'Sequence'])               
example_df

,Sequence header,Sequence
0,seq0,CCCGGGCCTTGTAGCACGCCGCCCGTTGTACCATGAATTGCATGCC...
1,seq1,TCCGGAGGGCGCTAAGCGAATGTAGGCTGATGACTGGGTGAAGTCG...
2,seq2,TGTGGCTGGATCACCTCCTTTCAGGGAGACCTACCCATACTGATAA...
3,seq3,TTTGATAGAGCGGTATCAGAGGTCAACTAGGTCGGTCGAGTCTTTC...
4,seq4,TTTGTCTGTTTAAATTGCATGAACCGGTACAGGTTGCGCACTGGCA...
5,seq5,GCGGCCTGGGCGGCGGAAATTACTGGCGTACCGGAAGCGCAGATAG...
6,seq6,ATACAAACTCCCTTTAAAATGGCCGATGGCACCTGTCTATGATAGG...
7,seq7,ATCCCACATAACACTATTGTATCGATACCTCGGCGGCGTAATTGCA...
8,seq8,AAGTGATTGTGGGGGTTTAGCTCAGTTGGTAGAGCGCCTGCTTTGC...
9,seq9,AACAACCCAGATCGTAAACCTGCGTACCAGGTTGAACGAAGCGCTC...


### 2. Реализация алгоритма КМП.

In [5]:
def prefix(s):
    """
    Computes prefix function for given string
    Returns: array with prefix function values
    """
    pi = [0 for _ in range(len(s))]
    for i in range(1, len(s)):
        j = pi[i-1]
        while j > 0 and s[i] != s[j]: 
            j = pi[j-1]
        if s[i] == s[j]:
            j += 1
        pi[i] = j
    return pi

def kmp(P,T):
    """
    Knut-Morris-Pratt allgorithm. Searches pattern P in string T
    Returns: array with match indexes
    """
    pl, tl = len(P), len(T)
    res = []
    p = prefix(P + "#" + T)
    count = 0
    for i in range(tl+1):
        if p[pl + i] == pl:
            count += 1
            res.append(i - pl + 1)
    return res

Пример использования на последовательности Arabidopsis, pattern='ATC' 

In [17]:
P = 'pattern_seq.fasta'
P_iter = fasta_iter(P)
for i in P_iter:
    header, P = i
print(header)    
print('Pattern:', P)    

Pattern_seq
Pattern: ATC


In [18]:
T = 'arabidopsis_atc.fasta'
T_iter=fasta_iter(T)
for i in T_iter:
    header, T = i
print(header)    
print('Template first 50 nucleotides:', T[:50])

AB024714.1 Arabidopsis thaliana ATC (centroradialis) gene, complete cds, strain:Columbia
Template first 50 nucleotides: CATAGGTTCGAAATTGGTTTATTATAATTTTAACGCACAAATTCCTTGTT


In [8]:
arr = kmp(P, T)

print('Match on indexes: ', ', '.join([str(i) for i in arr]))

Match on indexes:  82, 180, 223, 284, 504, 569, 573, 651, 700, 735, 805, 810, 929, 973, 1007, 1020, 1090, 1095, 1109, 1176, 1251, 1299, 1445, 1485, 1497, 1672, 1758, 1796, 1824, 1848, 1854, 1987, 2007, 2027, 2052, 2069, 2140, 2173, 2235, 2282, 2458, 2496, 2518, 2568, 2690, 2706, 2785, 2792, 2796, 2904, 2923, 3050


In [9]:
T[179:182]

'ATC'

Реализация одной фунцией, которую можно переиспользовать:

In [15]:
def search_fasta(pattern_fasta, template_fasta):
    P_iter = fasta_iter(pattern_fasta)
    for i in P_iter:
        _, P = i
    print('Pattern:', P)
    T_iter=fasta_iter(template_fasta)
    for i in T_iter:
        _, T = i
    print('Template first 50 nucleotides:', T[:50])
    return kmp(P, T)

In [16]:
result = search_fasta('pattern_seq.fasta', 'arabidopsis_atc.fasta')

print('Match on indexes: ', ', '.join([str(i) for i in result]))

Pattern: ATC
Template first 50 nucleotides: CATAGGTTCGAAATTGGTTTATTATAATTTTAACGCACAAATTCCTTGTT
Match on indexes:  82, 180, 223, 284, 504, 569, 573, 651, 700, 735, 805, 810, 929, 973, 1007, 1020, 1090, 1095, 1109, 1176, 1251, 1299, 1445, 1485, 1497, 1672, 1758, 1796, 1824, 1848, 1854, 1987, 2007, 2027, 2052, 2069, 2140, 2173, 2235, 2282, 2458, 2496, 2518, 2568, 2690, 2706, 2785, 2792, 2796, 2904, 2923, 3050


In [21]:
P=input()
T=input()
result = search_fasta(P, T)

print('Match on indexes: ', ', '.join([str(i) for i in result]))

pattern_seq.fasta
arabidopsis_atc.fasta
Pattern: ATC
Template first 50 nucleotides: CATAGGTTCGAAATTGGTTTATTATAATTTTAACGCACAAATTCCTTGTT
Match on indexes:  82, 180, 223, 284, 504, 569, 573, 651, 700, 735, 805, 810, 929, 973, 1007, 1020, 1090, 1095, 1109, 1176, 1251, 1299, 1445, 1485, 1497, 1672, 1758, 1796, 1824, 1848, 1854, 1987, 2007, 2027, 2052, 2069, 2140, 2173, 2235, 2282, 2458, 2496, 2518, 2568, 2690, 2706, 2785, 2792, 2796, 2904, 2923, 3050
